In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 28076367
paper_name = 'bozaquel_morais_montero_lomeli_2017' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [50]:
original_data = pd.read_excel('raw_data/journal.pone.0169682.s002.xlsx', sheet_name='PRIMARY HITS', skiprows=1)

In [51]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 157 x 7


In [52]:
original_data.head()

,SYSTEMATIC,NAME,SORAPHEN A 0.2 µg/ml,Unnamed: 3,SORAPHEN A 0.4 µg/ml,Unnamed: 5,TOTAL
0,NaN,NaN,SCORE A,SCORE B,SCORE A,SCORE B,NaN
1,YLR056W,ERG3,5,5,5,5,20.0
2,YBR289W,SNF5,5,5,5,5,20.0
3,YOL004W,SIN3,5,5,5,4,19.0
4,YAL013W,DEP1,4,5,5,4,18.0


In [53]:
original_data['orf'] = original_data['SYSTEMATIC'].astype(str)

In [54]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [55]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [56]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            SYSTEMATIC NAME SORAPHEN A 0.2 µg/ml Unnamed: 3  \
index_input                                                   
0                  NaN  NaN              SCORE A    SCORE B   

            SORAPHEN A 0.4 µg/ml Unnamed: 5  TOTAL  orf  
index_input                                              
0                        SCORE A    SCORE B    NaN  NAN  


In [57]:
original_data = original_data.loc[t,:]

In [58]:
original_data['data1'] = -original_data.iloc[:,2:4].sum(axis=1)
original_data['data2'] = -original_data.iloc[:,4:6].sum(axis=1)

In [59]:
original_data.set_index('orf', inplace=True)

In [60]:
original_data = original_data[['data1','data2']].copy()

In [61]:
original_data = original_data.groupby(original_data.index).mean()

In [62]:
original_data.shape

(156, 2)

# Load & process tested strains

In [63]:
tested = pd.read_excel('raw_data/Mat a collection YSC1053 open biosystems.xlsx', sheet_name='DATA')

In [64]:
tested.head()

,record no.,ORF name,Strain,Batch,Plate,Row,Col,Comment
0,338.0,YAL068C,BY4741,01_1,1.0,A,2.0,NaN
1,339.0,YAL067C,BY4741,01_1,1.0,A,3.0,NaN
2,340.0,YAL066W,BY4741,01_1,1.0,A,4.0,NaN
3,341.0,YAL065C,BY4741,01_1,1.0,A,5.0,NaN
4,345.0,YAL062W,BY4741,01_1,1.0,A,6.0,NaN


In [65]:
tested['orf'] = tested['ORF name'].astype(str)

In [66]:
tested['orf'] = clean_orf(tested['orf'])

In [67]:
tested.loc[tested['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [68]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [69]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record no. ORF name Strain Batch  Plate  Row  Col Comment  orf
index_input                                                                
4705                NaN      NaN    NaN   NaN    NaN  NaN  NaN     NaN  NAN


In [70]:
tested = tested.loc[t,:]

In [71]:
tested_orfs = np.unique(tested['orf'].values)

In [73]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [74]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

In [75]:
original_data.head()

,data1,data2
orf,,
YAL002W,0.0,0.0
YAL004W,0.0,0.0
YAL005C,0.0,0.0
YAL007C,0.0,0.0
YAL008W,0.0,0.0


# Prepare the final dataset

In [76]:
data = original_data.copy()

In [77]:
dataset_ids = [16260, 16261]
datasets = datasets.reindex(index=dataset_ids)

In [78]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [79]:
data.head()

dataset_id,16260,16261
data_type,value,value
orf,,
YAL002W,0.0,0.0
YAL004W,0.0,0.0
YAL005C,0.0,0.0
YAL007C,0.0,0.0
YAL008W,0.0,0.0


## Subset to the genes currently in SGD

In [80]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [81]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [82]:
data.head()

,dataset_id,16260,16261
,data_type,value,value
gene_id,orf,,
2,YAL002W,0.0,0.0
1863,YAL004W,0.0,0.0
4,YAL005C,0.0,0.0
5,YAL007C,0.0,0.0
6,YAL008W,0.0,0.0


# Normalize

In [83]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [84]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [85]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [86]:
data_all.head()


,dataset_id,16260,16261,16260,16261
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0.0,0.0,0.0,0.0
1863,YAL004W,0.0,0.0,0.0,0.0
4,YAL005C,0.0,0.0,0.0,0.0
5,YAL007C,0.0,0.0,0.0,0.0
6,YAL008W,0.0,0.0,0.0,0.0


# Print out

In [87]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [88]:
from IO.save_data_to_db3 import *

In [89]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 28076367...
Inserting the new data...


100%|██████████| 2/2 [00:14<00:00,  7.21s/it]

Updating the data_modified_on field...
